In [1]:
import tkinter as tk 
from tkinter.scrolledtext import *
import datetime
from tkinter_input_box.input_box import InputBox
from tkinter import Frame, Text, Scrollbar, Pack, Grid, Place
from tkinter.constants import RIGHT, LEFT, Y, BOTH

In [2]:
__all__ = ['ScrolledText']

# Make auto scroll without any scrollbar (code commented out)

class ScrolledText(Text):
    def __init__(self, master=None, **kw):
        self.frame = Frame(master)
        # self.vbar = Scrollbar(self.frame)
        # self.vbar.pack(side=RIGHT, fill=Y)

        # kw.update({'yscrollcommand': self.vbar.set})
        Text.__init__(self, self.frame, **kw)
        self.pack(side=LEFT, fill=BOTH, expand=True)
        # self.vbar['command'] = self.yview

        # Copy geometry methods of self.frame without overriding Text
        # methods -- hack!
        text_meths = vars(Text).keys()
        methods = vars(Pack).keys() | vars(Grid).keys() | vars(Place).keys()
        methods = methods.difference(text_meths)

        for m in methods:
            if m[0] != '_' and m != 'config' and m != 'configure':
                setattr(self, m, getattr(self.frame, m))

    def __str__(self):
        return str(self.frame)


def example():
    from tkinter.constants import END

    stext = ScrolledText(bg='white', height=10)
    stext.insert(END, __doc__)
    stext.pack(fill=BOTH, side=LEFT, expand=True)
    stext.focus_set()
    stext.mainloop()

In [3]:
def on_submit(event=None):
    user_input = input_field.get().lower()
   # User message (tag is user)
    output_field.config(state='normal')
    output_field.tag_config('user', foreground='white', background='#535353')
    output_field.insert('end', '\n' + user_input + '\n\n', 'user')
    
    # Chatbot response (tag is response)
    output_field.tag_config('response', foreground='white', background='#333333')
    output_field.insert('end', '\nBert: ', 'response')
    
    if user_input == 'hi':
        output_field.insert('end', 'Hello, how can I help you?\n\n', 'blue')
    elif user_input == 'how are you':
        output_field.insert('end', 'I am Okay. How about you?\n\n', 'blue')
    elif user_input == 'how old are you':
        output_field.insert('end', 'My creator just brought me to life in front of you. I am not much older than 10 minutes.\n\n', 'blue')
    elif user_input == 'what time is it':
        now = datetime.datetime.now()
        output_field.insert('end', "It's "+ now.strftime("%H:%M:%S %p")+'\n\n', 'blue')
    elif user_input == 'bye':
        output_field.insert('end', 'Goodbye, have a nice day!\n\n', 'blue')
    else:
        # output_field.tag_config('response', font=("Futura bold", 30))
        # output_field.insert('end', 'I LIKE PENIS\n\n', 'response')
        output_field.insert('end', 'I am sorry, I do not understand.\n\n', 'response')

    input_field.delete(0, 'end')
    output_field.yview(tk.END)
    output_field.config(state='disabled')

In [4]:
""" 
CODE TO USE WHEN GIVEN MODELS 

def on_submit(event=None):
    user_input = input_field.get().lower()
   # User message (tag is user)
    output_field.config(state='normal')
    output_field.tag_config('user', foreground='white', background='#535353')
    output_field.insert('end', '\n' + user_input + '\n\n', 'user')
    
    # Chatbot response (tag is response)
    output_field.tag_config('response', foreground='white', background='#333333')
    output_field.insert('end', '\nBert: ', 'response')
    
    
    # Get restaurant name recommendation 
    restaurant_recommendation = model_huy_insert_here(user_input)
    output_field.tag_config('response', foreground='white', background='#333333', font=("Futura bold", 15))
    output_field.insert('end', restaurant_recommendation, 'response')
    
    # Get restaurant details 
    restaurant_background = model_sophia_insert_here(user_input)
    output_field.tag_config('response', foreground='white', background='#333333', font=("Futura bold", 15))
    output_field.insert('end', restaurant_background, 'response')
    
    input_field.delete(0, 'end')
    output_field.yview(tk.END)
    output_field.config(state='disabled')

"""


' \nCODE TO USE WHEN GIVEN MODELS \n\ndef on_submit(event=None):\n    user_input = input_field.get().lower()\n   # User message (tag is user)\n    output_field.config(state=\'normal\')\n    output_field.tag_config(\'user\', foreground=\'white\', background=\'#535353\')\n    output_field.insert(\'end\', \'\n\' + user_input + \'\n\n\', \'user\')\n    \n    # Chatbot response (tag is response)\n    output_field.tag_config(\'response\', foreground=\'white\', background=\'#333333\')\n    output_field.insert(\'end\', \'\nBert: \', \'response\')\n    \n    \n    # Get restaurant name recommendation \n    restaurant_recommendation = model_huy_insert_here(user_input)\n    output_field.tag_config(\'response\', foreground=\'white\', background=\'#333333\', font=("Futura bold", 15))\n    output_field.insert(\'end\', restaurant_recommendation, \'response\')\n    \n    # Get restaurant details \n    restaurant_background = model_sophia_insert_here(user_input)\n    output_field.tag_config(\'response\

In [5]:
root = tk.Tk()
root.title("Eating")

''

In [6]:
# Output Frame 
output_frame = tk.Frame(root, width=150, height=500, relief = tk.FLAT, background =  "#65350F")
output_frame.pack_propagate(False)
output_frame.pack(side='top', fill='both', expand=True, padx=20)
#output_label = tk.Label(output_frame, text="Chatbot:")
#output_label.pack(side='left', padx=5, pady=5)

# Make autoscrolling when text is full 
output_field = ScrolledText(output_frame)

In [7]:
# Output Text details 
output_field.pack(side='left', fill='both', expand=True, padx=2, pady=3)
output_field.config(state='disabled')
output_field.tag_config('user', background='#1a3a46', justify='center')  # I don't think this does anything
output_field.tag_config('response', background='lightgreen', justify='center')  # I don't think this does anything 
output_field.config(font=("Futura", 15))
output_field.yview(tk.END)

In [8]:
# Input frame
input_frame = tk.Frame(root, width=125, height=20, relief = tk.FLAT, highlightbackground = "#65350F", highlightthickness = 2, background = "#EFEADD")
# input_frame.pack_propogate(False)
input_frame.pack(padx=20)
input_frame.pack(side='bottom', fill='x')
#input_label = tk.Label(input_frame, text="User:")
#input_label.pack(side='left', padx=5, pady=5)

In [9]:
input_field = tk.Entry(input_frame, width=125, bd=2.5, justify='center', relief = tk.FLAT, highlightbackground = "#EFEADD", highlightthickness=1, background = "#EFEADD")
# input_field = InputBox(root, placeholder="What type of food do you like?", font=("", 15))
# input_field.bind_all('<Key>', key_in)
input_field.pack(side='left', padx=5, pady=5)
input_field.bind("<Return>", on_submit)


'140476574272768on_submit'

In [10]:
root.mainloop()

In [ ]:
# Things to do: 
# 1) Consolidate code
# 2) try to make pre-emptive text
# 3) curved boxes 
# 4) make it look like texting back and forth 